In [2]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118
!pip install transformers accelerate bitsandbytes -q
!pip install -U bitsandbytes
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [77]:
from openai import OpenAI
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import Runnable
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [4]:
client = OpenAI(base_url='https://api.gapapi.com/v1', api_key='sk-9AtuC5pfE0lAZ2DEGdEzRsEgPXugkaJxBNefdBCSSXgE3G1S')

In [15]:
#def llm(prompt: str) -> str:
 #   resp = client.chat.completions.create(
  #      model="gpt-4o",
 #       messages=[{"role": "user", "content": prompt}]
 #   )
 #   resp = resp.choices[0].message.content
 #   resp = re.sub("\u200c", "", resp)
  #  return resp

In [16]:
#llm("سلام")

In [23]:
class SimpleLLM(Runnable):
    def invoke(self, prompt, *args, **kwargs) -> str:
        # اگر prompt دیکشنری باشد (از PromptTemplate)
        if not isinstance(prompt, str):
            if isinstance(prompt, dict):
                # فرض: فقط یک کلید وجود دارد (متن نهایی)
                prompt = next(iter(prompt.values()))
            else:
                prompt = str(prompt)

        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        text = resp.choices[0].message.content
        return re.sub("\u200c", "", text)

In [24]:
llm = SimpleLLM()

In [82]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

In [85]:
tools = [TavilySearchResults(max_results=1, tavily_api_key = "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")]

In [87]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("Latest news about Elon Musk.")
print(response)



> Entering new AgentExecutor chain...
Thought: I should use the tavily_search_results_json tool to find the latest news about Elon Musk by inputting an appropriate search query.  
Action: tavily_search_results_json  
Action Input: "latest news about Elon Musk"  
Observation: [{'title': "Elon Musk News | Today's Latest Stories - Reuters", 'url': 'https://www.reuters.com/business/elon-musk/', 'content': "# Elon Musk\n\n## Latest Stories\n\nTesla shares recouped all the losses they suffered during the public feud, when more than $150 billion was wiped off the company's market value.\n\nU.S. President Donald Trump and Elon Musk attend a press conference at the White House\n\nPresident Donald Trump received a phone call from Elon Musk on Monday before the billionaire expressed regret over some of the posts he made on social media last week, according to a White House source with knowledge of the matter. [...] Tesla and its CEO Elon Musk are walking an increasingly difficult tightrope as t

In [91]:
response = agent.run("today news")
print(response)



> Entering new AgentExecutor chain...
To answer the question about today's news, I would need to use the search tool optimized for current event information.

Action: tavily_search_results_json  
Action Input: today's news
Observation: [{'title': 'CNN: Breaking News, Latest News and Videos', 'url': 'https://www.cnn.com/', 'content': "WASHINGTON, DC - JULY 18: U.S. President Donald Trump delivers remarks during a signing ceremony for the “GENIUS Act” in the East Room of the White House July 18, 2025 in Washington, DC. The act, formally known as the Guiding and Establishing National Innovation for U.S. Stablecoins Act, was passed this week by the U.S. Congress.. (Photo by Win McNamee/Getty Images) [...] President of France Emmanuel Macron and his wife Brigitte Macron arrive at Royal Palace 'Huis ten Bosch' for the social dinner for Heads of State and Government during the 76th NATO Summit in the World Forum in The Hague, Netherlands on June 24th, 2025.\n\n## US news\n\nShow all\n\nKilm